# Yahoo
marco

[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TFMUCM01/TFM/blob/main/Data%20IBEX%2035/ibex35yahoo1.ipynb)


#Exportacion de libreria

In [ ]:
!pip install investpy

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
from datetime import datetime
import textwrap
import requests
import yfinance as yf
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

#Explicacion de las variables

📈 Ratios de valoración
PER (Price/Earnings): Cuántas veces los beneficios netos están contenidos en el precio actual de la acción. Un valor alto puede indicar sobrevaloración.

Forward PER: Igual que el PER, pero usando las ganancias proyectadas del próximo año.

PEG Ratio: Relación entre PER y el crecimiento esperado de beneficios. Un PEG < 1 puede indicar una acción infravalorada.

Price/Sales: Relación entre el precio de la acción y las ventas por acción. Útil cuando no hay beneficios.

Price/Book: Compara el precio de la acción con el valor contable por acción. < 1 puede sugerir infravaloración.

🏦 Rentabilidad
Return on Equity (ROE): Rentabilidad sobre el capital propio (%). Muestra cuán eficientemente la empresa usa sus recursos propios.

Return on Assets (ROA): Rentabilidad sobre activos totales (%). Indica cuán rentable es la empresa respecto a todos sus recursos.

Gross Margins: Margen bruto (%). Beneficio después del coste de ventas, antes de otros gastos.

Operating Margins: Margen operativo (%). Beneficio operativo respecto a las ventas.

Profit Margins: Margen neto (%). Beneficio neto sobre ingresos. Mide la rentabilidad final.

💳 Solvencia y liquidez
Debt/Equity: Relación entre deuda total y patrimonio. Mide el apalancamiento financiero.

Current Ratio: Activos corrientes / Pasivos corrientes. Liquidez a corto plazo (>1 es generalmente bueno).

Quick Ratio: Como el Current Ratio pero excluyendo inventarios. Más conservador.

⚖️ Riesgo y dividendos
Beta: Volatilidad relativa respecto al mercado. >1 implica más riesgo; <1 implica menos.

Dividend Yield: Rentabilidad por dividendo (%). Mide cuánto paga la empresa en dividendos en relación con el precio de su acción.

💰 Tamaño y valor de la empresa
Market Cap (M): Capitalización bursátil en millones de euros. Valor total de la empresa en bolsa.

Enterprise Value (M): Valor total de la empresa considerando deuda y caja. Útil para valorar compras.

📊 Magnitudes financieras clave (últimos 12 meses - TTM)
Revenue (TTM): Ingresos totales.

Net Income (TTM): Beneficio neto.

EBITDA: Beneficio antes de intereses, impuestos, depreciación y amortización. Indicador operativo.

Free Cash Flow: Flujo de caja libre. Dinero disponible después de inversiones necesarias.

Total Debt: Deuda total.

Total Cash: Caja y equivalentes disponibles.

🧑‍💼 Datos estructurales
Number of Employees: Número de empleados a tiempo completo.

Sector: Sector económico donde opera la empresa.

Industry: Industria específica.

#DataFrame

In [ ]:
import investpy
import pandas as pd

# Lista de países
paises = ['germany', 'france', 'united kingdom', 'netherlands', 'switzerland', 'sweden', 'spain', 'italy']

# Diccionario de sufijos Yahoo Finance
sufijos_yahoo_filtrados = {
    'germany': '.DE',
    'france': '.PA',
    'united kingdom': '.L',
    'netherlands': '.AS',
    'switzerland': '.SW',
    'sweden': '.ST',
    'spain': '.MC',
    'italy': '.MI'
}

todos = []

for pais in paises:
    try:
        # No usas alias aquí, si quieres, puedes definirlo
        df = investpy.get_stocks(country=pais)
        df['country'] = pais
        df['tickets_y'] = df['symbol'] + sufijos_yahoo_filtrados.get(pais, '')
        df = df.rename(columns={'symbol': 'ticket'})
        todos.append(df[['ticket', 'tickets_y', 'name', 'country']])
    except Exception as e:
        print(f"❌ Error con {pais}: {e}")

# Unir todos los resultados
df_todos = pd.concat(todos, ignore_index=True)
df_todos.to_csv('tickers_europa.csv', index=False)

print(f"✅ Guardados {len(df_todos)} tickers europeos con encabezados ['ticket', 'tickets_y', 'name', 'country'] en 'tickers_europa.csv'")


In [ ]:
import pandas as pd

# Cargar CSV (ajusta el nombre si es diferente)
df = pd.read_csv('tickers_europa.csv')

# Obtener la columna 'tickets_y' como lista, eliminando nulos
tickers = df['tickets_y'].dropna().unique().tolist()

print(tickers)


In [ ]:
import time
import yfinance as yf
import pandas as pd

ratios_dict = {}

ratios_keys = {
    "PER (Price/Earnings)": "trailingPE",
    "Forward PER": "forwardPE",
    "PEG Ratio": "pegRatio",
    "Price/Sales": "priceToSalesTrailing12Months",
    "Price/Book": "priceToBook",
    "Return on Equity": "returnOnEquity",
    "Return on Assets": "returnOnAssets",
    "Debt/Equity": "debtToEquity",
    "Current Ratio": "currentRatio",
    "Quick Ratio": "quickRatio",
    "Gross Margins": "grossMargins",
    "Operating Margins": "operatingMargins",
    "Profit Margins": "profitMargins",
    "Beta": "beta",
    "Dividend Yield": "dividendYield",
    "Market Cap (M)": "marketCap",
    "Enterprise Value (M)": "enterpriseValue",
    "Revenue (TTM)": "totalRevenue",
    "Net Income (TTM)": "netIncomeToCommon",
    "EBITDA": "ebitda",
    "Free Cash Flow": "freeCashflow",
    "Total Debt": "totalDebt",
    "Total Cash": "totalCash",
    "Number of Employees": "fullTimeEmployees",
    "Sector": "sector",
    "Industry": "industry"
}

for ti in tickers:
    retry = 3
    while retry > 0:
        try:
            ticker = yf.Ticker(ti)
            info = ticker.info
            ratios_data = {}

            for key, field in ratios_keys.items():
                value = info.get(field, None)

                if isinstance(value, (int, float)) and field in ["marketCap", "enterpriseValue", "totalRevenue", "netIncomeToCommon", "ebitda", "freeCashflow", "totalDebt", "totalCash"]:
                    value = round(value / 1_000_000, 2)
                ratios_data[key] = value

            ratios_dict[ti] = ratios_data
            print(f"✅ {ti} procesado correctamente.")
            break  # salir del while al completar exitosamente

        except Exception as e:
            msg = str(e).lower()
            if "too many requests" in msg or "rate limit" in msg:
                retry -= 1
                wait_time = 10
                print(f"⚠️ Rate limit detectado para {ti}. Esperando {wait_time} segundos. Intentos restantes: {retry}")
                time.sleep(wait_time)
            else:
                print(f"❌ Error al procesar {ti}: {e}")
                break

    time.sleep(1)  # espera entre tickers para evitar bloqueos

df_ratios = pd.DataFrame.from_dict(ratios_dict, orient='index')
df_ratios.index.name = "Empresa"
df_ratios = df_ratios[sorted(df_ratios.columns)]
df_ratios.to_csv('ratios.csv', index=False)
print("✅ Archivo guardado como 'ratios.csv'")

ERROR:yfinance:HTTP Error 404: 


✅ BASFn.DE procesado correctamente.


ERROR:yfinance:HTTP Error 404: 


✅ BAYGn.DE procesado correctamente.
✅ ALVG.DE procesado correctamente.


ERROR:yfinance:HTTP Error 404: 


...

✅ RXPA.PA procesado correctamente.
✅ TFFP.PA procesado correctamente.


ERROR:yfinance:HTTP Error 404: 


✅ TEPRF.PA procesado correctamente.
✅ CRIP.PA procesado correctamente.
✅ SDGI.PA procesado correctamente.
✅ EPMF.PA procesado correctamente.
✅ KOF.PA procesado correctamente.
✅ SOPR.PA procesado correctamente.
✅ RUBF.PA procesado correctamente.
✅ ADUX.PA procesado correctamente.
✅ PARP.PA procesado correctamente.
✅ SONO.PA procesado correctamente.
✅ ASY.PA procesado correctamente.
✅ GECI.PA procesado correctamente.
✅ BOND.PA procesado correctamente.
✅ STF.PA procesado correctamente.
✅ SWOR.PA procesado correctamente.
✅ AVOM.PA procesado correctamente.
✅ XPO.PA procesado correctamente.
✅ UFFP.PA procesado correctamente.
✅ CNIM.PA procesado correctamente.
✅ LAEP.PA procesado correctamente.
✅ VEIL.PA procesado correctamente.
✅ AVQ.PA procesado correctamente.
✅ LOUP.PA procesado correctamente.
✅ PARRO.PA procesado correctamente.
✅ TFF.PA procesado correctamente.
✅ CITT.PA procesado correctamente.
✅ ALHRG.PA procesado correctamente.
✅ IPAR.PA procesado correctamente.
✅ CGR.PA procesado corr

ERROR:yfinance:HTTP Error 404: 


✅ MED6.PA procesado correctamente.
✅ GRBT.PA procesado correctamente.
✅ IISP.PA procesado correctamente.
✅ ALBOU.PA procesado correctamente.
✅ VRKP.PA procesado correctamente.
✅ OPEN.PA procesado correctamente.
✅ SAMS.PA procesado correctamente.
✅ MBRI.PA procesado correctamente.
✅ LAFU.PA procesado correctamente.
✅ GLTN.PA procesado correctamente.
✅ BAIN.PA procesado correctamente.


In [ ]:
df_ratios = df_ratios.reset_index()
df_ratios

# Ranking

In [ ]:
df_ratios_ibex35["Rank_ROE"] = df_ratios_ibex35["Return on Equity"].rank(ascending=False, method="min")
df_ratios_ibex35["Rank_FCF"] = df_ratios_ibex35["Total Cash"].rank(ascending=False, method="min")
df_ratios_ibex35["Rank_ProfitMargin"] = df_ratios_ibex35["Profit Margins"].rank(ascending=False, method="min")
df_ratios_ibex35["Rank_Total"] = df_ratios_ibex35[["Rank_ROE", "Rank_FCF", "Rank_ProfitMargin"]].mean(axis=1)
df_ratios_ibex35_sorted = df_ratios_ibex35.sort_values("Rank_Total")
print(df_ratios_ibex35_sorted[["Ticker", "Return on Equity", "Total Cash", "Profit Margins", "Rank_Total"]].head(10))

In [ ]:
plt.figure(figsize=(16,5))

plt.subplot(1,3,1)
sns.barplot(data=df_ratios_ibex35_sorted.head(10), x="Ticker", y="Return on Equity")
plt.xticks(rotation=45)
plt.title("Top 10 Empresas por ROE")

plt.subplot(1,3,2)
sns.barplot(data=df_ratios_ibex35_sorted.head(10), x="Ticker", y="Total Cash")
plt.xticks(rotation=45)
plt.title("Top 10 Empresas por Total Cash")

plt.subplot(1,3,3)
sns.barplot(data=df_ratios_ibex35_sorted.head(10), x="Ticker", y="Profit Margins")
plt.xticks(rotation=45)
plt.title("Top 10 Empresas por Profit Margins")

plt.tight_layout()
plt.show()
